# NER Tagging

In [1]:
import spacy
nlp = spacy.load('en')

In [26]:
sent_0 = nlp(u'Donald Trump visited at the government headquarters in France today.')
sent_1 = nlp(u'Emmanuel Jean-Michel Frédéric Macron is a French politician serving as President of France and ex officio Co-Prince of Andorra since 14 May 2017.')
sent_2 = nlp(u"He studied philosophy at Paris Nanterre University, completed a Masters of Public Affairs at Sciences Po and graduated from the Ecole Nationale Administration ( ENA ) in 2004.")
sent_3 = nlp(u'He worked at the Inspectorate General of Finances, and later became an investment banker at Rothschild & Cie Banque.')

In [7]:
for token in sent_0:
    print((token.text, token.ent_type_))

('Donald', 'PERSON')
('Trump', 'PERSON')
('visited', '')
('at', '')
('the', '')
('government', '')
('headquarters', '')
('in', '')
('France', 'GPE')
('today', 'DATE')
('.', '')


In [8]:
for ent in sent_0.ents:
    print((ent.text, ent.label_))

('Donald Trump', 'PERSON')
('France', 'GPE')
('today', 'DATE')


In [10]:
for token in sent_1:
    print((token.text, token.ent_type_))

('Emmanuel', 'PERSON')
('Jean', 'PERSON')
('-', 'PERSON')
('Michel', 'PERSON')
('Frédéric', 'PERSON')
('Macron', 'PERSON')
('is', '')
('a', '')
('French', 'NORP')
('politician', '')
('serving', '')
('as', '')
('President', '')
('of', '')
('France', 'GPE')
('and', '')
('ex', '')
('officio', '')
('Co', 'LOC')
('-', 'LOC')
('Prince', 'LOC')
('of', 'LOC')
('Andorra', 'LOC')
('since', '')
('14', 'DATE')
('May', 'DATE')
('2017', 'DATE')
('.', '')


In [11]:
for ent in sent_1.ents:
    print((ent.text, ent.label_))

('Emmanuel Jean-Michel Frédéric Macron', 'PERSON')
('French', 'NORP')
('France', 'GPE')
('Co-Prince of Andorra', 'LOC')
('14 May 2017', 'DATE')


In [27]:
for token in sent_2:
    print((token.text, token.ent_type_))

('He', '')
('studied', '')
('philosophy', '')
('at', '')
('Paris', 'ORG')
('Nanterre', 'ORG')
('University', 'ORG')
(',', '')
('completed', '')
('a', '')
('Masters', 'ORG')
('of', 'ORG')
('Public', 'ORG')
('Affairs', 'ORG')
('at', '')
('Sciences', 'PERSON')
('Po', 'PERSON')
('and', '')
('graduated', '')
('from', '')
('the', 'ORG')
('Ecole', 'ORG')
('Nationale', 'ORG')
('Administration', 'ORG')
('(', 'ORG')
('ENA', 'ORG')
(')', '')
('in', '')
('2004', 'DATE')
('.', '')


In [28]:
for ent in sent_2.ents:
    print((ent.text, ent.label_))

('Paris Nanterre University', 'ORG')
('Masters of Public Affairs', 'ORG')
('Sciences Po', 'PERSON')
('the Ecole Nationale Administration ( ENA', 'ORG')
('2004', 'DATE')


In [29]:
for token in sent_3:
    print((token.text, token.ent_type_))

('He', '')
('worked', '')
('at', '')
('the', 'ORG')
('Inspectorate', 'ORG')
('General', 'ORG')
('of', 'ORG')
('Finances', 'ORG')
(',', '')
('and', '')
('later', '')
('became', '')
('an', '')
('investment', '')
('banker', '')
('at', '')
('Rothschild', 'ORG')
('&', 'ORG')
('Cie', 'ORG')
('Banque', 'ORG')
('.', '')


In [34]:
for ent in sent_3.ents:
    print((ent.text, ent.label_))

('the Inspectorate General of Finances', 'ORG')
('Rothschild & Cie Banque', 'ORG')


# Training NER-Tagger

In [36]:
import random
from pathlib import Path
import spacy

In [35]:
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]

In [38]:
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == '__main__':
    main()

Created blank 'en' model
{'ner': 12.199768738465764}
{'ner': 11.513408303260803}
{'ner': 15.573740243911743}
{'ner': 11.35499358177185}
{'ner': 10.973198054301747}
{'ner': 9.411028406186672}
{'ner': 8.649543368980577}
{'ner': 6.390611172650413}
{'ner': 4.647946692830654}
{'ner': 5.947591517907044}
{'ner': 4.3240432240106585}
{'ner': 3.5617322492489656}
{'ner': 3.8325075377099513}
{'ner': 6.087161831590495}
{'ner': 5.87361541650935}
{'ner': 3.0081661968533178}
{'ner': 5.450491458907945}
{'ner': 4.782577121783454}
{'ner': 0.011780621182735434}
{'ner': 2.637216401222697}
{'ner': 3.693213673991523}
{'ner': 2.062314101794708}
{'ner': 1.2336936068781257}
{'ner': 1.9985481408451848}
{'ner': 0.9933587312698368}
{'ner': 0.010272905720546684}
{'ner': 1.943981082357273}
{'ner': 6.777601073564012e-10}
{'ner': 4.231733765828193e-10}
{'ner': 1.9640578031541216}
{'ner': 2.3289289733617304e-14}
{'ner': 0.002388502017144102}
{'ner': 7.793507125548667e-08}
{'ner': 1.0148375034332275}
{'ner': 0.011213295

Example of training an additional entity type
This script shows how to add a new entity type to an existing pre-trained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.
The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.
After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.
For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities
Compatible with: spaCy v2.0.0+

In [42]:
LABEL = 'ANIMAL'
TRAIN_DATA = [
    ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'ANIMAL')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'ANIMAL')]
    })
]

In [43]:
def main(model=None, new_model_name='animal', output_dir=None, n_iter=20):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    ner.add_label(LABEL)   # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()



    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                           losses=losses)
            print(losses)

    # test the trained model
    test_text = 'Do you like horses?'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)


if __name__ == '__main__':
    main()

Created blank 'en' model
{'ner': 32.08625324785015}
{'ner': 7.327082341743982}
{'ner': 4.014112027122389}
{'ner': 1.941191638966806}
{'ner': 5.188534382147202}
{'ner': 2.1804696872675716e-08}
{'ner': 1.709496386388387}
{'ner': 2.000000004300269}
{'ner': 4.3942900795221047e-07}
{'ner': 5.675983793450685e-11}
{'ner': 1.0282989332521685e-06}
{'ner': 0.00024232443012479728}
{'ner': 1.3095552429218662e-14}
{'ner': 1.763662133395246e-08}
{'ner': 0.00027838388449496934}
{'ner': 7.2245108139340714e-12}
{'ner': 2.801903138173593e-11}
{'ner': 1.614592796453162}
{'ner': 6.254393728426677e-24}
{'ner': 6.181376252261e-20}
Entities in 'Do you like horses?'
ANIMAL horses
